In [1]:
%pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from docling.document_converter import DocumentConverter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document as LCDocument
from langchain_core.document_loaders import BaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import Iterator

In [3]:
class DoclingPDFLoader(BaseLoader):

    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [4]:
folder_path = "../../docs"
files = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
loader = DoclingPDFLoader(file_path=files)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

In [5]:
docs = loader.load()
splits = text_splitter.split_documents(docs)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
len(splits)

41

In [7]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [8]:
vectordb = FAISS.from_documents(splits, embeddings_model)
print(vectordb.index.ntotal)

41


In [9]:
!rm -rf summarizer_index
vectordb.save_local("summarizer_index")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
